# Problem Set 1

#### Set up environment

In [2]:
# Set working directory
dir = "/Users/JoshuaHigbee/Box/2. Second Year/2. Winter Quarter - 2021/" *
      "Industrial Organization II - Hortacsu/Problem Sets/Problem Set 1/";
cd(dir);

# Load packages
using CSV, DataFrames, Random, Distributions, LinearAlgebra,
      LatexPrint, StatsBase, Plots, SpecialFunctions
using Optim, ForwardDiff, PyCall

# Set seed
Random.seed!(12345);

# Read in data
data = CSV.read("psetOne.csv", DataFrame);
println(names(data))

["Market", "Constant", "Price", "EngineSize", "SportsBike", "Brand2", "Brand3", "z1", "z2", "z3", "z4", "shares"]


<br/><br/><br/><br/><br/><br/>

## Question 8

#### Load data for market $t=17$, and set up parameter values (coefficient order is Price, Constant, Engine CC, BikeType, Brand2, Brand3) and $\xi$ values.

In [3]:
# Data 
d17 = data[data.Market .== 17, :];

# Parameters
θ_given =  [-3.0 1.0 1.0 2.0 -1.0 1.0]'
α = θ_given[1]

# Ownership matrix
d17.Brand1 = Ref(1.0) .- max.(d17.Brand2, d17.Brand3);
Δ = (d17.Brand1 .* d17.Brand1') .+ (d17.Brand2 .* d17.Brand2') .+ (d17.Brand3 .* d17.Brand3')

# ξ values
ξ_rand = rand(Normal(0,1),7);
ξ_newton = ξ_rand; # ξ_simple
ξ_newton = zeros(7);

#### Define useful functions for solving fixed point pricing equation.

In [4]:
# Shares function
function s_newton(price)
    X = hcat(price, d17.Constant, d17.EngineSize, d17.SportsBike, d17.Brand2, d17.Brand3)
    exp_term = X * θ_given .+ ξ_newton
    max_exp = maximum(exp_term)
    num = exp.(exp_term .- max_exp)
    return num / (exp(-max_exp) + sum(num))
end


# Jacobian of shares function
function s_Jac_newton(price)
    s = s_newton(price)
    return - α * ((s * s') .- diagm(vec(s))) # α is negative in this setting
end


# Hessian of shares function
function s_Hess_newton(price)
    s = s_newton(price)
    Hess = Array{Float64}(undef, 7, 7, 7)
    for j in 1:7
        for k in 1:7
            for l in 1:7
                Hess[j,k,l] = α^2 *
                      ((2*s[j]*s[k]*s[l] - 3*s[j]*s[k] + s[j])*(j==k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[l])*(j==k && j!=l) +
                      (2*s[j]*s[k]*s[l] - s[k]*s[l])*(j!=k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[k])*(j!=k && k==l) +
                      (2*s[j]*s[k]*s[l])*(l!=k && k!=j && j!=l))
            end
        end
    end
    return Hess
end


# Function for Newton's method (using p + Ω^{-1} * s = 0)
function f_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    f = price + inv(Ω) * s
    return f
end


# Jacobian of Ω^{-1} (for use in computing Jacobian of function for fxp est)
function Ω_inv_Jac_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    s_H = s_Hess_newton(price)
    Ω = Δ .* s_J
    Ω_J = Δ .* s_H

    Ω_J_inv = Array{Float64}(undef, length(s), length(s), length(s))
    for j in 1:length(s)
        Ω_J_inv[:,:,j] = - inv(Ω) * Ω_J[:,:,j] * inv(Ω)
    end
    return Ω_J_inv
end


# Jacobian of function for Newton's method
function f_newton_J(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    Ω_inv_J = Ω_inv_Jac_newton(price)
    Ω_inv_J_times_s = Matrix(0.0 * I, length(s), length(s))
    for j in 1:length(s)
        Ω_inv_J_times_s[:,j] = Ω_inv_J[:,:,j] * s
    end
    ∇f_newton = I + Ω_inv_J_times_s + inv(Ω) * s_J
    return ∇f_newton
end


# Actually estimate the fixed point
function newton_fxp(price, ϵ)
    diff = 1
    p_0 = price
    iter=1
    while diff > ϵ
        # Solve equation
        fxp = f_newton(p_0)
        ∇fxp = f_newton_J(p_0)
        # ∇fxp = ForwardDiff.jacobian(f_newton, p_0)
        p_1 = p_0 - inv(∇fxp) * fxp

        # Update values and loop
        diff = maximum(abs.(p_1 .- p_0))
        println("Iteration " * string(iter) * "     Difference " * string(diff))
        p_0 = p_1
        iter = iter + 1
    end
    return p_0
end

newton_fxp (generic function with 1 method)

#### Solve fixed point equation.

In [5]:
p_init = ones(7);
p_init = d17.Price;
p_optimal = newton_fxp(p_init, 1.0e-10)

println(" ")
p_optimal

Iteration 1     Difference 2.484826536163365
Iteration 2     Difference 0.2893243176990721
Iteration 3     Difference 0.0726965625358531
Iteration 4     Difference 0.006524393561025077
Iteration 5     Difference 4.2079754713597595e-5
Iteration 6     Difference 1.679278494037817e-9
Iteration 7     Difference 6.661338147750939e-16
 


7×1 Array{Float64,2}:
 0.38447413297903904
 0.3844741329790391
 0.3523194580438232
 0.3523194580438232
 1.7822602606218438
 1.782260260621844
 1.782260260621844

#### Check to ensure it is a fixed point.

In [6]:
f_newton(p_optimal)
show(stdout, "text/plain", f_newton(p_optimal))

7×1 Array{Float64,2}:
 -1.1102230246251565e-16
  0.0
  0.0
 -5.551115123125783e-17
  3.9968028886505635e-15
  3.9968028886505635e-15
  3.3306690738754696e-15

<br/><br/><br/><br/><br/><br/>

## Question 9

#### Write share prediction functions.

In [7]:
# # Individual share predictor function (given mean utilities)
# function sHat_ind(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    
#     # Fill values if unspecified (default)
#     if X == 0.0
#         X = repeat(fill(0.0,length(σ))', length(δ))
#     end
#     if ζ == 0.0
#         ζ = reshape(fill(0.0,length(σ)*I_n), I_n, length(σ))
#     end

#     # Construct numerators
#     const_term = repeat(δ', outer=I_n)
#     rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
#     exp_term = const_term .+ rc_term
#     max_exp = maximum(exp_term, dims=2)
#     num = exp.(exp_term - repeat(max_exp, inner=(1,length(δ))))

#     # Construct denominators and shares
#     denom = repeat(exp.(-max_exp) .+ sum(num, dims=2), inner=(1,length(δ)))
#     ŝ_i = num ./ denom
#     return ŝ_i
# end

# Individual share predictor function (given mean utilities)
function sHat_ind(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    
    # Fill values if unspecified (default)
    if X == 0.0
        X = repeat(fill(0.0,length(σ))', length(δ))
    end
    if ζ == 0.0
        ζ = reshape(fill(0.0,length(σ)*I_n), I_n, length(σ))
    end

    # Construct numerators
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    num = exp.(exp_term)

    # Construct denominators and shares
    denom = repeat(Ref(1.0) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end

# Share predictor function (for heterogeneity)
function sHat(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    ŝ_i = sHat_ind(δ=δ, X=X, σ=σ, ζ=ζ, I_n=I_n)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

# Share predictor function for taking ŝ after it's already computed
function sHat_from_ŝ(; ŝ_i, I_n=1)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

sHat_from_ŝ (generic function with 1 method)

#### Set up parameters and data for test cases.

In [8]:
J_q9 = [3, 3, 3];
δ_q9 = [zeros(J_q9[1]), [40, 20, 20], zeros(J_q9[3])];
σ_q9 = [[0.0], [0.0], [0.1]];
X_q9 = [[], [], [repeat([1.0], 3), [1.0; 3.0; -1.0], [10.0; 10.0; -3.0]]];
I_q9 = [[], [], 50];
ζ_q9 = [[], [], reshape(rand(Normal(0,1),length(σ_q9[3])*I_q9[3]), I_q9[3], length(σ_q9[3]))];
shares_q9 = [];

#### Test all cases

In [9]:
# First case
s_test = sHat(δ=δ_q9[1], σ=σ_q9[1]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.25, 0.25, 0.25]


In [10]:
# Second case
s_test = sHat(δ=δ_q9[2], σ=σ_q9[2]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.9999999958776928, 2.0611536139418496e-9, 2.0611536139418496e-9]


In [11]:
# Third case
s_test = sHat(δ=δ_q9[3], X=X_q9[3][1], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][2], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][3], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

[0.2499492, 0.2499492, 0.2499492]
[0.247299, 0.2547225, 0.2504074]
[0.2492555, 0.2492555, 0.2680796]


<br/><br/><br/><br/><br/><br/>

## Question 10 - - - - - - - CURRENTLY HAS TWO CONTRACTION MAPPING FUNCTIONS

#### Create functions for share inversion.

In [12]:
# # Contraction mapping function
# function s_contraction_map(;s, X, I_n, σ, ζ, J, L̄=1.0-1.0e-7, output=false)
#       δ_0 = convert.(Real, zeros(length(s)))
#       L = 0.5 # Initialize contraction mapping ratio
#       iter = 0 # Initialize iteration count
#       δ_diff = 0.5 # Initialize difference in δ (in case L misbehaves)
#       δ_diff_h = Array{Real}(undef, 1, 3)
#       while (δ_diff > 1.0e-7) && (L < L̄)
#             # Numerical inversion
#             δ_1 = δ_0 .+ log.(s) .- log.(sHat(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n))

#             # Max distance in δ
#             δ_diff = maximum(abs.(δ_1 - δ_0))

#             # Track iterations and update Lipschitz constant every 50 iters
#             iter = iter + 1
#             if ((iter + 2) % 50) == 0
#                   δ_diff_h[1] = Real(δ_diff)
#             end
#             if ((iter + 1) % 50) == 0
#                   δ_diff_h[2] = Real(δ_diff)
#             end
#             if (iter % 50) == 0
#                   δ_diff_h[3] = Real(δ_diff)
#                   L = (δ_diff_h[2]^2) / (δ_diff_h[3] * δ_diff_h[1])
#                   if output==true
#                         println("Iteration  " * string(iter) *
#                               "    Contraction term L is " * string(L))
#                   end
#             end
#             δ_0 = δ_1
#       end

#       return δ_0
# end


# Contraction mapping function
function s_contraction_map(;s, X, I_n, σ, ζ, J, ϵ_contract = 1.0e-5, output=false)
      δ_0 = convert.(Real, zeros(length(s)))
      iter = 0 # Initialize iteration count
      δ_diff = 0.5 # Initialize difference in δ (in case L misbehaves)
      δ_diff_h = Array{Real}(undef, 1, 3)
      while (δ_diff > ϵ_contract) 
            # Numerical inversion
            δ_1 = δ_0 .+ log.(s) .- log.(sHat(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n))

            # Max distance in δ
            δ_diff = maximum(abs.(δ_1 - δ_0))

            if (iter % 50) == 0 && output==true
                println("Iteration  " * string(iter) * "    Difference is " * string(δ_diff))
            end
            δ_0 = δ_1
      end

      return δ_0
end


# Log shares Jacobian function - vectorized
function log_s_Jac(; ŝ_i, I_n=1, J)
      ∫ŝ_j_mat = repeat(1/(I_n) .* sum(ŝ_i, dims=1), outer=J)'
      own_term = 1.0/(I_n) * ŝ_i' * (Ref(1.0) .- ŝ_i) .* Matrix(I, J, J)
      cross_term = -1.0/(I_n) * (ŝ_i' * ŝ_i) .* (Ref(1.0) .- Matrix(I, J, J))
      return (own_term .+ cross_term) ./ ∫ŝ_j_mat
end


# Newton's method function
function s_newton_fxp(; s, δ_0, X, I_n, σ, ζ, J, ϵ_conv=1.0e-14, output=false, maxiter = 1e5)
    diff = 0.5
    iter = 0 # Initialize iteration count
    while diff > ϵ_conv && iter < maxiter
        # Compute
        ŝ_i = sHat_ind(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n)
        ŝ = sHat_from_ŝ(ŝ_i=ŝ_i, I_n=I_n)
        log_sJ = log_s_Jac(ŝ_i=ŝ_i, I_n=I_n, J=J)
        log_s = log.(ŝ)
        δ_1 = δ_0 .- inv(log_sJ) * (log_s - log.(s))

        # Check difference and iterate
        diff = maximum(abs.(δ_1 .- δ_0))
        iter = iter + 1
        if output==true
              println("Iteration  " * string(iter) *
                    "    Difference is " * string(diff))
        end
        δ_0 = δ_1
    end

    return δ_0
end


# Combine two methods function
function invert_s(; s, X, I_n, σ, ζ, J, ϵ_contract=1.0e-5, ϵ_conv=1.0e-14, output=false, maxiter_newton = 1e5)
    δ_contraction = s_contraction_map(s=s, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, ϵ_contract=ϵ_contract, output=output)
    δ = s_newton_fxp(s=s, δ_0=δ_contraction, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, ϵ_conv=ϵ_conv, output=output, 
            maxiter = maxiter_newton)
    return δ
end

invert_s (generic function with 1 method)

#### Test functions with data from market 17.

In [13]:
I_17 = 25;
X_17 = convert(Matrix, d17[:, [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]]);
Z_17 = convert(Matrix, d17[:, [:z1, :z2, :z3, :z4]]);
σ_17 = zeros(size(X_17)[2])';
ζ_17 = reshape(rand(Normal(0,1),length(σ_17)*I_17), I_17, length(σ_17));

In [14]:
# Test contraction
δ_contract = s_contraction_map(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1])

7-element Array{Float64,1}:
  2.0697845385676574
 -0.3439439517828262
 -2.9273567831528453
  0.6068451864947533
  3.141160653013621
  1.3883038447921399
  2.3472954727255972

In [15]:
# Test Newton's method
s_newton_fxp(s=d17.shares, δ_0 = δ_contract, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], ϵ_conv=1.0e-14)

7-element Array{Float64,1}:
  2.070264979033944
 -0.34346351131653924
 -2.9268763426865583
  0.6073256269610395
  3.1416410934799073
  1.3887842852584265
  2.3477759131918825

# CONTRACTION EPSILON IS LARGE HERE

In [16]:
# Test both
invert_s(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], ϵ_contract=1.0e-4, ϵ_conv=1.0e-14)

7-element Array{Float64,1}:
  2.070264979033939
 -0.34346351131654446
 -2.926876342686563
  0.6073256269610349
  3.1416410934799024
  1.3887842852584218
  2.347775913191879

#### Test with data from the previous question.

In [17]:
println(invert_s(s=shares_q9[1], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[1]))
println(invert_s(s=shares_q9[2], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[2], ϵ_contract=1.0e-4))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][1], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][2], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][3], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))

[0.0, 0.0, 0.0]
[38.46485052094403, 18.464850520944033, 18.464850520944033]
[0.0, 0.0, 0.0]
[0.00016785577094455723, -0.029659081493348612, -0.012089398619323165]
[-0.0611761787212328, -0.0611761787212328, -0.14113460280480175]


<br/><br/><br/><br/><br/><br/>

## Question 11

#### Write function for computing $\xi$

In [18]:
# Implicit function for ξ - all markets (X_all is vector of matrices)
function ξ_rclogit(; X_all, θ, s_all, ζ=0.0, I_num=1, maxiter_newton=1e5, rclogit_output=false)
    # θ split between linear terms (1) and nonlinear terms (2)
    αβ = θ[1] # Put price first for consistency
    σ = θ[2]
    
    # Loop through markets and invert shares
    δhat_all = []
    for mkt in 1:length(X_all)
        if rclogit_output==true
            println("Market number is " * string(mkt) * " of " * string(length(X_all)))
        end
        δhat = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]), 
                    ϵ_contract=1.0e-3, ϵ_conv=1.0e-14, maxiter_newton = maxiter_newton)
        push!(δhat_all, δhat)
    end
    
    # Stack data and compute δ_mean
    δhat = reduce(vcat, δhat_all)
    X_stack = reduce(vcat, X_all)
    δ_mean = X_stack * αβ

    # Subtract off mean utility to get ξ̂
    return δhat .- δ_mean
end

ξ_rclogit (generic function with 1 method)

#### Write function for splicing data into markets (array of matrices and vectors, for shares and variables).

In [19]:
function mkt_X_s_break(; exclude_vec)
    X_full_mkt = []
    s_full_mkt = []
    for mkt in unique(data[[x ∉ exclude_vec for x in data.Market], :Market])
        X_mkt = convert(Matrix, data[data.Market .== mkt,
                    [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]])
        push!(X_full_mkt, X_mkt)
        
        s_mkt = data[data.Market .== mkt, :shares]
        push!(s_full_mkt, s_mkt)
    end
    return [X_full_mkt, s_full_mkt]
end

mkt_X_s_break (generic function with 1 method)

#### Test functions with data from market.

In [110]:
# Create test parameters and select data
I_test = 40
σ_test = zeros(6)
ζ_test = reshape(rand(Normal(0,1),length(σ_test)*I_test), I_test, length(σ_test))
Z_test = convert(Matrix, data[:, [:z1, :z2, :z3, :z4]])
W_test = inv(Z_test' * Z_test);


# Test with expanded instrument set
X_no_p = reduce(vcat, test_data[1])[:,2:6]
Z_full = hcat(X_no_p, Z_test)
W_full = inv(Z_full' * Z_full)


# Choose data
test_data = mkt_X_s_break(exclude_vec = []);

In [69]:
# Test functions at θ = 0
θ_zeros = [zeros(6), zeros(6)']
@time ξ_zeros = ξ_rclogit(X_all=test_data[1], θ=θ_zeros, s_all=test_data[2], ζ=ζ_test, I_num=I_test);

 24.965661 seconds (127.18 M allocations: 17.186 GiB, 20.53% gc time)


In [70]:
ξ_zeros[1:8]

8-element Array{Float64,1}:
  3.417557706918591
  2.2199898323356098
 -2.8348206236877678
  4.317272874619474
  1.459620984134135
  1.0254328494996232
  1.5368650848736685
 -2.7318744786927707

#### Write GMM objective function using implicit function $\xi$.

In [23]:
function f_gmm_rclogit_implθ(; W=I, Z, X_all, θ, s_all, ζ, I_num)
    ξ = ξ_rclogit(X_all=X_all, θ=θ, s_all=s_all, ζ=ζ, I_num=I_num)
    return (Z' * ξ)' * W * (Z' * ξ)
end

# Test objective function - computing ξ within (only z1-z4)
f_gmm_rclogit_implθ(W=W_test, Z=Z_test, X_all=test_data[1], θ=[zeros(6), zeros(6)'], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

252.59328031304517

In [54]:
# Test objective function - computing ξ within (using full instruments, including characteristics)
f_gmm_rclogit_implθ(W=W_full, Z=Z_full, X_all=test_data[1], θ=[zeros(6), zeros(6)'], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

2947.137997931572

#### Rewrite objective function as a function of only $\sigma$ (no linear terms)

In [80]:
# Function for computing linear parameters
function θ_tsls(; X, Z, W, δ)
    B = X' * Z * W * Z'
    return inv(B * X) * (B * δ)
end


# Function returning ξ(σ), δ(σ), and θ̄(σ)
function ξδθ_σ_rclogit(; X_all, σ, s_all, ζ=0.0, I_num=1, Z, W)
    δ = []
    for mkt in 1:length(X_all)
        δ_mkt = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]))
        push!(δ, δ_mkt)
    end
    δ_σ_stack = reduce(vcat, δ)
    X_stack = reduce(vcat, X_all)

    αβ = θ_tsls(X=X_stack, Z=Z, W=W, δ=δ_σ_stack)
    
    ξ_all = []
    for mkt in 1:length(X_all)
        ξ_mkt = δ[mkt] .- X_all[mkt] * αβ
        push!(ξ_all, ξ_mkt)
    end
    ξ = reduce(vcat, ξ_all)
    
    return (ξ = ξ, δ_all = δ, αβ = αβ, ξ_all = ξ_all)
end


# Rewrite objective function as only a function of σ
function f_gmm_rclogit_σ(;W, Z, X_all, σ, s_all, ζ, I_num)
    output = ξδθ_σ_rclogit(X_all=X_all, σ=σ, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W)
    ξ = output.ξ
    # println(output.αβ)
    return (Z' * ξ)' * W * (Z' * ξ)
end

f_gmm_rclogit_σ (generic function with 1 method)

In [26]:
# Test objective function at σ=0
f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], ζ=ζ_test, 
    I_num=I_test)

18.878411530341467

<br/><br/><br/><br/>
#### The objective. Let’s get values for markets t = 1 and t = 17, with sigma = 0, and Z is all the market Z.

In [27]:
# Test objective function at σ=0 (only market 1)
f_gmm_rclogit_σ(W=inv(Z_test' * Z_test), Z=convert(Matrix, data[data.Market.==1, [:z1, :z2, :z3, :z4]]), 
    X_all=[test_data[1][1]], σ=zeros(6)', 
    s_all=[test_data[2][1]], ζ=ζ_test, I_num=I_test)

0.002350557910935882

In [30]:
@time f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

 28.036022 seconds (167.65 M allocations: 25.109 GiB, 20.46% gc time)


0.7241370155056548

<br/><br/><br/><br/><br/><br/>

## Question 12

#### Write gradient function (as a function of $\sigma$ only).

In [90]:
output = ξδθ_σ_rclogit(X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], ζ=ζ_test, I_num=I_test,
    Z=Z_test, W=W_test);

In [97]:
shat_test = sHat_ind_αβ(αβ = output.αβ, X=test_data[1][1], ξ=output.ξ_all[1], σ=ones(6)',
    ζ=ζ_test, I_n=I_test);

In [99]:
# Individual share predictor function (given X and αβ)
function sHat_ind_αβ(; αβ, X, ξ, σ, ζ=0.0, I_n=1)
    # Construct numerators
    δ = X * αβ .+ ξ
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    num = exp.(exp_term)

    # Construct denominators and shares
    denom = repeat(Ref(1.0) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end


# Function to compute Jacobian of ξ(θ) for each market
function ξ_Jac_θ_mkt(; X, I_n, ζ, ŝ_i)
    ξ_J = zeros(size(X)[1], size(X)[1])
    J_σ = zeros(size(X)[1], size(X)[2])
    for i in 1:I_n
        s = ŝ_i[i,:]
        ξ_Ji = -1.0 * ((s * s') .- diagm(vec(s)))
        ξ_J = ξ_J .+ (1/I_n) .* ξ_Ji
        J_σ = J_σ .+ (ξ_Ji * X) * diagm(ζ[i,:])
    end
    return -1.0 * inv(ξ_J) * J_σ./I_n
end


# Function for computing Jacobian of ξ(θ) for all markets
function ξ_Jac(; X_all, αβ, I_n, ζ, σ, ξ_all)
      ξ_J_θ = []
      for mkt in 1:length(X_all)
            X_mkt = X_all[mkt];
            ŝ_i = sHat_ind_αβ(αβ=αβ, X=X_mkt, ξ=ξ_all[mkt], σ=σ, ζ=ζ, I_n=I_n)
            ξ_J_θ_mkt = ξ_Jac_θ_mkt(X=X_mkt, I_n=I_n, ζ=ζ, ŝ_i=ŝ_i)
            push!(ξ_J_θ, ξ_J_θ_mkt)
      end
      ξ_J = reduce(vcat, ξ_J_θ)
      return ξ_J
end

ξ_Jac (generic function with 1 method)

#### Write functions that compute analytic gradient (and objective function as a function of $\sigma$ only, for comparison with ForwardDiff).

In [100]:
# Function that computes gradient given σ and only σ
function f_gmm_rclogit_σ_g(; X_all, s_all, Z, W, σ, I_n, ζ)
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ, s_all=s_all, ζ=ζ, I_num=I_n, Z=Z, W=W);
    ξ_J = ξ_Jac(X_all=X_all, αβ=ξδθ_output.αβ, I_n=I_n, ζ=ζ, σ=σ, ξ_all=ξδθ_output.ξ_all)
    df_θ = 2 * (Z' * ξ_J)' * W * (Z' * ξδθ_output.ξ)
    return df_θ
end

# Function that computes objective function for given σ, using full data
function gmm_σ_test_full(σ_init) 
    func = f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=σ_init,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

# Function that computes objective function for given σ, using "test" data
function gmm_σ_test(σ_init) 
    func = f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=σ_init,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ_test (generic function with 1 method)

#### Compare auto-differentiation with analytic gradients.

##### Using z1-z4:

At 0:

In [101]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_test, W=W_test, 
    σ=zeros(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 -282.907  20.6999  -537.221  -22.84  3.7184  -28.162

In [34]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test, zeros(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 0.957584  18.7794  34.0614  0.501048  -0.295995  -0.16443

At 0.1:

In [106]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_test, W=W_test, 
    σ=0.1*ones(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 -259.388  101.306  917.877  -23.2381  -14.3924  -90.4662

In [36]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test, 0.1*ones(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 61.257  -5.28611  453.086  12.2696  -5.54666  -0.864887

##### Using expanded instrument set:

At 0:

In [103]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
    σ=zeros(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 -2.06635e-12  -2.27511e-12  1.49341e-11  …  -1.44035e-13  1.96688e-12

In [38]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test_full, zeros(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 -6.15277e-15  -5.13637e-15  3.93809e-14  …  2.09548e-15  -3.40415e-15

At 0.1:

In [115]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
    σ=0.1*ones(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 0.142389  0.0297017  1.24713  0.0441153  0.144087  0.20602

In [116]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test_full, 0.1*ones(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 0.142389  0.0297017  1.24713  0.0441153  0.144087  0.20602

Discussion?

<br/><br/><br/><br/><br/><br/>

## Question 13

#### Write GMM function that only takes in the desired values of $\sigma$ (for random coefficients on price and engine size).

##### GMM function using only z1-z4.

In [41]:
function gmm_σ(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    func = f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=σ_mod,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ (generic function with 1 method)

In [107]:
# Test GMM function
gmm_σ(0.1*ones(2)')

3.9382951920911218

In [60]:
function gmm_σ_g(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    grad = f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_test, W=W_test, 
                    σ=σ_mod, I_n=I_test, ζ=ζ_test)'
    return grad[[1,3]]
end

gmm_σ_g (generic function with 1 method)

In [108]:
gmm_σ_g(0.1*ones(2)')

2-element Array{Float64,1}:
   66.42296148132766
 -233.03681165127614

##### GMM function using full sets of instruments.

In [48]:
function gmm_σ_full(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    func = f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=σ_mod,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ_full (generic function with 1 method)

In [109]:
# Test GMM function
gmm_σ_full(0.1*ones(2)')

0.772801277628305

In [57]:
function gmm_σ_full_g(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    grad = f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
                σ=σ_mod, I_n=I_test, ζ=ζ_test)'
    return grad[[1,3]]
end

gmm_σ_full_g (generic function with 1 method)

In [117]:
gmm_σ_full_g(0.1*ones(2)')

2-element Array{Float64,1}:
 0.14995226877601997
 1.0197988287607602

##### One more sanity check to make sure the gradients match.

In [118]:
# Construct auto-diff function as placeholder
g_gmm_σ_auto_full = x -> ForwardDiff.gradient(gmm_σ_full, x)
g_gmm_σ_auto_full(0.1 * ones(2)')

1×2 Adjoint{Float64,Array{Float64,1}}:
 0.149952  1.0198

#### Write function for calling one stage of GMM for a given weighting matrix.

In [119]:
function gmm_stage(; W, Z, X_all, s_all, ζ, I_num, σ_init)
    # Objective function
    function f_obj(σ_init) 
        σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
        func = f_gmm_rclogit_σ(W=W, Z=Z, X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num)
        return func
    end
    
    # Gradient function
    function g_obj(G, σ_init)
        grad = gmm_σ_full_g(σ_init)
        G[1:2] = grad
    end
    
#     # AD gradient function - for testing Optim and comparison with coded gradient function
#     g_gmm_σ_auto_test = x -> ForwardDiff.gradient(f_obj, x)
#     function g_obj(G, σ_init)
#         grad = g_gmm_σ_auto_test(σ_init)
#         G[1:2] = grad[1:2]
#     end
    
    # Return result
    σ_result = Optim.minimizer(optimize(f_obj, g_obj, σ_init, BFGS(), Optim.Options(show_trace=true)))
    return σ_result
end  

gmm_stage (generic function with 1 method)

In [120]:
gmm_stage(W=W_full, Z=Z_full, X_all=test_data[1], s_all=test_data[2], ζ=ζ_test, I_num=I_test, 
    σ_init=0.1*ones(2))

Iter     Function value   Gradient norm 
     0     7.760773e-01     1.019799e+00
 * time: 0.0031409263610839844
     1     7.567699e-01     7.298156e-01
 * time: 32.961628913879395
     2     7.261348e-01     1.434374e-01
 * time: 117.93993782997131
     3     7.241460e-01     8.346210e-03
 * time: 162.37639093399048
     4     7.241370e-01     3.220799e-04
 * time: 238.96201491355896
     5     7.241370e-01     7.185621e-07
 * time: 286.306275844574
     6     7.241370e-01     5.592656e-11
 * time: 366.7638268470764


2-element Array{Float64,1}:
 -2.026765828312821e-11
  7.808585295611604e-12

In [125]:
# gmm_stage(W=W_full, Z=Z_full, X_all=test_data[1], s_all=test_data[2], ζ=ζ_test, I_num=I_test, 
#     σ_init=0.5*ones(2))

In [152]:
# Choose step size
function step_size(; x_1, x_0, g_1, g_0)
    # x's are columns (so transpose is row), g's are columns
    num = (x_1 .- x_0) * (g_1 .- g_0)'
    denom = (g_1[1] - g_0[1])^2 + (g_1[2] - g_0[2])^2
    return abs((num / denom)[1])
end

step_size (generic function with 1 method)

In [189]:
# Gradient descent function
function descent(; σ, start_step, ϵ_conv)
    
    # Gradient
    g_0 = gmm_σ_full_g(σ) 
    
    # Prepare for iterating
    step = start_step; iter = 0
    x_0 = σ
    x_1 = x_0 .- start_step * g_0
    g_1 = gmm_σ_full_g(x_1) 
    
    println(g_1)
    println(g_0)
    
    # Value of objective
    obj = gmm_σ_full(x_1) 
    println("Iteration is " * string(iter) * "    Objective function is " * string(obj))
    println("   Step size was " * string(step))
    println("   Parameter values are " * string(x_1))
    
    # Iterate
    while abs(step) > ϵ_conv
        step = step_size(x_1=x_1, x_0=x_0, g_1=g_1, g_0=g_0)
        if step != NaN
            x_new = x_1 .- step * g_1
            g_new = gmm_σ_full_g(x_1)

            # Update
            x_0 = copy(x_1); x_1 = copy(x_new);
            g_0 = copy(g_1); g_1 = copy(g_new);
            println(g_new)
            println(g_1)
            println(g_0)

            iter = iter + 1
            obj = gmm_σ_full(x_1) 
            println("Iteration is " * string(iter) * "    Objective function is " * string(obj))
            println("   Step size was " * string(step))
            println("   Parameter values are " * string(x_1))
        else
            step = ϵ_conv/2
        end
    end
    return (σ=x_1, obj_val=obj)
end

descent (generic function with 1 method)

In [190]:
test = descent(σ=0.5*ones(2), start_step=0.05, ϵ_conv=1e-6)

[1.6355103430904085, -17.419294425540144]
[-0.1585550891091021, 19.614250024137906]
Iteration is 0    Objective function is 4.091711214330781
   Step size was 0.05
   Parameter values are [0.5079277544554551, -0.48071250120689535]
[1.6355103430904085, -17.419294425540144]
[1.6355103430904085, -17.419294425540144]
[1.6355103430904085, -17.419294425540144]
Iteration is 1    Objective function is 4.08854513735258
   Step size was 1.0346176364886647e-5
   Parameter values are [0.5079108331769989, -0.48053227811461685]
[1.6358417854405882, -17.408642813349292]
[1.6358417854405882, -17.408642813349292]
[1.6355103430904085, -17.419294425540144]
Iteration is 2    Objective function is NaN
   Step size was NaN
   Parameter values are [NaN, NaN]


(σ = [NaN, NaN], obj_val = NaN)

In [128]:
(zeros(2) .- ones(2))'

1×2 Adjoint{Float64,Array{Float64,1}}:
 -1.0  -1.0

#### Write functions for implementing two-stage GMM.

In [ ]:
# Function for optimal weighting matrix
function gmm_opt_weight(; Z, ξ)
    n = size(Z)[1]
    Zξ = Z' * ξ
    return inv((1/n) * Zξ * Zξ')
end


# Function for gradient of ξ with respect to all parameters - STILL NEED TO FINISH THIS!!!!!!!!!!!!!!!!!!!!!!!
function ZJ_θfull_J(; Z, ξ, X_all, αβ, I_n, ζ, σ)
    n = size(Z)[1]
    Jac_αβ = reduce(vcat, X_all)
    Jac_σ = ξ_Jac(X_all=X_all, αβ=αβ, I_n=I_n, ζ=ζ, σ=σ)
    Jac_θ = hcat(Jac_αβ, Jac_σ)
    return -(1/n) * Z' * Jac_θ
end


# Function for two-stage GMM
function two_stage_gmm(; W_stage1, Z, X_all, s_all, ζ, I_num, σ_init)
    # First stage estimation with given starting weighting matrix
    σ_stage1 = gmm_stage(W=W_stage1, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_init)
    
    # Compute optimal weighting matrix for second stage
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_stage1, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage1)
    ξ_stage1 = ξδθ_output.ξ
    W_stage2 = gmm_opt_weight(Z=Z, ξ=ξ_stage1)
    
    # Second stage estimation with given starting weighting matrix
    σ_stage2 = gmm_stage(W=W_stage2, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_stage1)
    
    # Compute covariance matrix for point estimates
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_stage2, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage2)
    G = ZJ_θfull_J(Z=Z, ξ=.ξδθ_outputξ, X_all=X_all, αβ=ξδθ_output.αβ, I_n=I_num, ζ=ζ, σ=σ)
    ξ_stage1 = ξδθ_output.ξ
    W_stage2 = gmm_opt_weight(Z=Z, ξ=ξ_stage1)
    Σ = inv(G' * inv(W_stage2) * G)
    
    # Return values
    return (σ = σ_stage2, αβ = ξδθ_output.αβ, Σ = Σ)
end

<br/><br/><br/><br/><br/><br/><br/><br/>

## Question 14

##### Use PyBLP

In [191]:
pyblp = pyimport("pyblp")
np = pyimport("numpy")
pd = pyimport("pandas")
os = pyimport("os")

PyObject <module 'os' from '/Users/JoshuaHigbee/.julia/conda/3/lib/python3.8/os.py'>

In [192]:
os.getcwd()

"/Users/JoshuaHigbee/Box/2. Second Year/2. Winter Quarter - 2021/Industrial Organization II - Hortacsu/Problem Sets/Problem Set 1"

In [193]:
product_data = pd.read_csv("psetOne.csv")

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2,z3,z4,shares
0,1,1,1.554708,3.0,1,0,0,0.237685,0.063116,0.034361,0.056459,0.239077
1,1,1,1.559811,5.0,1,0,0,0.002033,0.186073,0.326445,0.062091,0.072184
2,1,1,1.857186,3.0,1,1,0,0.164550,0.033308,0.075316,0.278736,0.000460
3,1,1,1.995438,5.0,1,0,1,0.485538,0.117025,0.345850,0.084891,0.587867
4,1,1,4.017987,7.5,1,0,1,0.582503,0.368032,0.652134,0.509693,0.033746
...,...,...,...,...,...,...,...,...,...,...,...,...
1042,150,1,2.010710,3.0,1,1,0,0.257544,0.068546,0.095527,0.185333,0.000548
1043,150,1,2.001591,7.5,0,1,0,0.042531,0.474048,0.285886,0.263406,0.024166
1044,150,1,1.692977,5.0,1,0,1,0.021038,0.453095,0.079379,0.276368,0.355868
1045,150,1,2.337965,7.5,1,0,1,0.565742,0.190674,0.695795,0.044509,0.347937


In [194]:
product_data.rename(columns=Dict("Price"=>"prices", "Market"=>"market_ids", "z1"=>"demand_instruments0",
        "z2"=>"demand_instruments1", "z3"=>"demand_instruments2", "z4"=>"demand_instruments3"), inplace="True")

In [195]:
product_formulations = (pyblp.Formulation("prices + 1 + EngineSize + SportsBike + Brand2 + Brand3"),
   pyblp.Formulation("0 + prices + EngineSize"))

(PyObject 1 + prices + EngineSize + SportsBike + Brand2 + Brand3, PyObject prices + EngineSize)

In [196]:
mc_integration = pyblp.Integration("monte_carlo", size=40, specification_options=Dict("seed"=>12345))

PyObject Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 12345}.

In [197]:
problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration, add_exogenous="True")

PyObject Dimensions:
 T    N     I     K1    K2    MD 
---  ----  ----  ----  ----  ----
150  1047  6000   6     2     9  

Formulations:
       Column Indices:           0         1           2           3         4       5   
-----------------------------  ------  ----------  ----------  ----------  ------  ------
 X1: Linear Characteristics      1       prices    EngineSize  SportsBike  Brand2  Brand3
X2: Nonlinear Characteristics  prices  EngineSize                                        

In [198]:
bfgs = pyblp.Optimization("bfgs", Dict("gtol"=>1e-6))

PyObject Configured to optimize using the BFGS algorithm implemented in SciPy with analytic gradients and options {gtol: +1.000000E-06}.

In [199]:
results = problem.solve(sigma=np.identity(2), optimization=bfgs)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     I     K1    K2    MD 
---  ----  ----  ----  ----  ----
150  1047  6000   6     2     9  

Formulations:
       Column Indices:           0         1           2           3         4       5   
-----------------------------  ------  ----------  ----------  ----------  ------  ------
 X1: Linear Characteristics      1       prices    EngineSize  SportsBike  Brand2  Brand3
X2: Nonlinear Characteristics  prices  EngineSize                                        
Solving the problem ...

Nonlinear Coefficient Initial Values:
  Sigma:       prices       EngineSize  
----------  -------------  -------------
  prices    +1.000000E+00               
EngineSize  +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
  Sigma:       prices       EngineSize  
----------  -------------  -------------
  prices        -INF                    
EngineSize  +0.000000E+00      -INF     

Nonlinear Co

PyObject Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +1.136687E+02  +3.525152E+01  -1.230461E+08   +2.091842E+09      0      +2.146586E+03      +3.351864E+04  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:33       No           4            40         120666       368602   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
  Sigma:        prices         EngineSize   
----------  ---------------  ---------------
  prices     +3.311047E-02        

In [206]:
αβ_pyblp = results.beta

6×1 Array{Float64,2}:
  1.2935728843531908
 -1.9694667377821897
  0.5882426601881855
  0.6240084675211102
 -0.9345504046789983
  0.42864047278026596

In [207]:
σ_pyblp = diag(results.sigma)

2-element Array{Float64,1}:
  0.03311047248382437
 -0.0062606797775226

<br/><br/><br/><br/><br/><br/><br/><br/>

## Question 15

#### Elasticity matrix for market $t=17$.

#### Get predicted $\xi$ from the parameters found earlier

In [213]:
σ_pyblp_all = [σ_pyblp[1], 0, σ_pyblp[2], 0, 0, 0]'
αβ_pyblp_all = [αβ_pyblp[2], αβ_pyblp[1], αβ_pyblp[3:6]]

output = ξδθ_σ_rclogit(X_all=test_data[1], σ=σ_pyblp_all, s_all=test_data[2], ζ=ζ_test, I_num=I_test,
    Z=Z_full, W=W_full)

ξ_17 = output.ξ_all[17];

In [216]:
function elasticity(; αβ, X, σ, ζ, ξ, prices)
    # Get necessary numbers
    J = size(X,1)
    el = zeros(J,J)
    n = size(ζ)
    
    # Predict individual and aggregate shares of each good
    ŝ_i = sHat_ind_αβ(αβ=αβ, X=X, ξ=ξ, σ=σ, ζ=ζ, I_n=n)
    s_mkt = mean(ŝ_i, dims=2)
    
    # Vector of individual price coefficients
    α_i = Ref(αβ[1]) .+ ζ * σ[1]
    
    # Fill in elasticity matrix
    for j in 1:J
        for k in 1:J
            if j==k
                el[j,k] = - (prices[j]/s_mkt[j]) * mean(α_i .* ŝ_i[i,j]*(1-ŝ_i[i,j]) for i in 1:n)
            else
                el[j,k] = (prices[k]/s_mkt[j]) * mean(α_i .* ŝ_i[i,j]*ŝ_i[i,k] for i in 1:n)
            end
        end
    end
    return el
end

elasticity (generic function with 3 methods)

In [218]:
Elast = elasticity(αβ=αβ_pyblp, X=X_17, σ=σ_pyblp_all, ζ=ζ_test, ξ=ξ_17, 
    prices=X_17[1,:])

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 42 and 7")

In [ ]:
lap(Elast)